# TP1
## Question 2: ZLW

In [2]:
import numpy as np

### File loading and dictionary initialization

In [3]:
FILES = [f"texte_{i}.txt" for i in range(1,6)]
PATH = "../data/textes/"

def load_file(index: int) -> str:
	if index < 0 or index >= len(FILES):
		raise ValueError("Index out of range")
	with open(PATH+FILES[index],"r") as f:
		msg = f.read()
	return msg

def init_dict(msg: str) -> dict[str, str]:
	dict_symb = {}
	n_symb = 0
	for i in range(len(msg)):
		if msg[i] not in dict_symb:
			dict_symb[msg[i]] = f"{n_symb:b}"
			n_symb +=1
	return dict_symb

### Utils

In [4]:
def dict_update_size(dict_symb: dict[str, str]):
	n_symb = len(dict_symb)
	for k, v in dict_symb.items():
		dict_symb[k] = v.zfill(int(np.ceil(np.log2(n_symb))))

log_size = lambda x: np.ceil(np.log2(x))


### Compression algorithm

In [5]:
def compress(msg: str, dict_symb: dict[str, str]) -> (list[str], int):
	i, length = 0, 0
	n_symb = len(dict_symb)
	coded_msg = []
	while i < len(msg):
		# Next coded string
		next_str = msg[i]
		# Same, but with extra character (for the dictionary)
		next_str_extra = msg[i]

		# Tries to fit the largest string possible in the dictionary
		while next_str_extra in dict_symb and i < len(msg):
			i += 1
			next_str = next_str_extra
			if i < len(msg): # If there is still characters to read
				next_str_extra += msg[i]

		# Coding of the string
		bin_code = dict_symb[next_str]
		coded_msg.append(bin_code)
		length += len(bin_code)
		# Adding the new string to the dictionary
		if i < len(msg):
			dict_symb[next_str_extra] = f"{n_symb:b}"
			n_symb += 1

		# Updating symbols size if necessary
		if log_size(n_symb) > len(coded_msg[-1]):
			dict_update_size(dict_symb)
	return coded_msg, length

In [8]:
import time
def run(index: int, verbose: bool = True) -> dict[str, str]:
	start = time.time()
	msg = load_file(index)
	loaded = time.time()
	if verbose:
		print(f"Message: {msg}")
	dict_symb = init_dict(msg)
	initial_length = int(log_size(len(dict_symb))*len(msg))
	dict_update_size(dict_symb)
	dict_loaded = time.time()
	if verbose:
		print(f"Binary symbols: {dict_symb}")
		print(f"Initial length: {initial_length}")
	coded_msg, length = compress(msg, dict_symb)
	compressed = time.time()
	print(f"Coded message: {coded_msg}")
	print(f"Or : {''.join(coded_msg)}")

	if verbose:
		print(f"Final dictionary: {dict_symb}")
		print("")
		print(f"Length = {length}")
		print(f"Original length = {initial_length}")
		print(f"Compression rate = {100 - length/initial_length*100:.2f}%")
		print(f"Compression factor = {initial_length/length:.2f}")

	if verbose:
		print("")
		print(f"Loading time: {(loaded-start)*10**3:.2f}ms")
		print(f"Dictionary loading time: {(dict_loaded-loaded)*10**3:.2f}ms")
		print(f"Compression time: {(compressed-dict_loaded)*10**3:.2f}ms")
		print(f"Total time: {(compressed-start)*10**3:.2f}ms")
	return dict_symb
dict_symb = run(0)


Message: AAAAAAAAAAAAAAAAABBABAAAAAAAAAAAABAAAAAAAAAAAAAABBAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAA
Binary symbols: {'A': '0', 'B': '1'}
Initial length: 84
Coded message: ['0', '10', '11', '100', '101', '010', '001', '0001', '0000', '1001', '0110', '0101', '1011', '1100', '1101', '01110', '01101', '01111', '10011', '01111']
Or : 01011100101010001000100001001011001011011110011010111001101011111001101111
Final dictionary: {'A': '00000', 'B': '00001', 'AA': '00010', 'AAA': '00011', 'AAAA': '00100', 'AAAAA': '00101', 'AAAAAA': '00110', 'AAB': '00111', 'BB': '01000', 'BA': '01001', 'AB': '01010', 'BAA': '01011', 'AAAAAAA': '01100', 'AAAAAB': '01101', 'BAAA': '01110', 'AAAAAAAA': '01111', 'AAAAABB': '10000', 'BAAAA': '10001', 'AAAAABA': '10010', 'AAAAAAAAA': '10011', 'AAAAAAAAAA': '10100'}

Length = 74
Original length = 84
Compression rate = 11.90%
Compression factor = 1.14

Loading time: 0.85ms
Dictionary loading time: 0.11ms
Compression time: 0.32ms
Total time: 1.28ms


In [9]:
for k, v in dict_symb.items():
    print(f"{k} : {v}")

A : 00000
B : 00001
AA : 00010
AAA : 00011
AAAA : 00100
AAAAA : 00101
AAAAAA : 00110
AAB : 00111
BB : 01000
BA : 01001
AB : 01010
BAA : 01011
AAAAAAA : 01100
AAAAAB : 01101
BAAA : 01110
AAAAAAAA : 01111
AAAAABB : 10000
BAAAA : 10001
AAAAABA : 10010
AAAAAAAAA : 10011
AAAAAAAAAA : 10100
